### **HuggingFace Login**

In [1]:
from huggingface_hub import login

login()

### **Import Libairies**

In [2]:
from IPython.display import display

import numpy as np
import pandas as pd
from tqdm import tqdm
import torch

from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import Dataset

### **Read Datasets**

In [3]:
"""
df_train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv", index_col=0)
df_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv", index_col=0)

df_train = pd.read_csv("/media/yanncauchepin/ExternalDisk/Datasets/NaturalLanguageProcessing/disaster_tweets/train.csv", index_col=0)
df_test = pd.read_csv("/media/yanncauchepin/ExternalDisk/Datasets/NaturalLanguageProcessing/disaster_tweets/test.csv", index_col=0)
"""

df_train = pd.read_csv(
    "C:/Users/cauchepy/Datasets/NaturalLanguageProcessing/kaggle_disastertweets/train.csv",
    index_col=0,
)
df_test = pd.read_csv(
    "C:/Users/cauchepy/Datasets/NaturalLanguageProcessing/kaggle_disastertweets/test.csv",
    index_col=0,
)

### **Short Analysis**

In [5]:
print(f"Length - train {len(df_train)} - test {len(df_test)}")

### **Preprocess Datasets**

##### _Merge columns (full)_

In [6]:
keywords = pd.concat(
    [
        pd.DataFrame(df_train["keyword"].value_counts()).rename(
            columns={"count": "train"}
        ),
        pd.DataFrame(df_test["keyword"].value_counts()).rename(
            columns={"count": "test"}
        ),
    ],
    axis=1,
)
keywords.head()

,train,test
keyword,,
fatalities,45,5
deluge,42,8
armageddon,42,8
sinking,41,9
damage,41,9


In [7]:
locations = pd.concat(
    [
        pd.DataFrame(df_train["location"].value_counts()).rename(
            columns={"count": "train"}
        ),
        pd.DataFrame(df_test["location"].value_counts()).rename(
            columns={"count": "test"}
        ),
    ],
    axis=1,
)
locations.head()

,train,test
location,,
USA,104.0,37.0
New York,71.0,38.0
United States,50.0,15.0
London,45.0,13.0
Canada,29.0,13.0


In [8]:
df_train_full = df_train.copy()
df_train_full["text"] = df_train_full.apply(
    lambda row: f"{row['location'] or ''} {row['keyword'] or ''} {row['text']}".strip(),
    axis=1,
)
df_test_full = df_test.copy()
df_test_full["text"] = df_test_full.apply(
    lambda row: f"{row['location'] or ''} {row['keyword'] or ''} {row['text']}".strip(),
    axis=1,
)

### **Model Assessment**

In [9]:
def evaluate_classifier(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average="weighted")
    precision = precision_score(y_true, y_pred, average="weighted")
    recall = recall_score(y_true, y_pred, average="weighted")
    cm = confusion_matrix(y_true, y_pred)

    metrics_df = pd.DataFrame(
        {"Value": [f1, precision, recall]}, index=["F1 Score", "Precision", "Recall"]
    )

    cm_df = pd.DataFrame(
        cm,
        columns=["Predicted Negative", "Predicted Positive"],
        index=["Actual Negative", "Actual Positive"],
    )

    return metrics_df, cm_df

### **BERT Tokenizer**

##### _BERT Tokenizer + split validation_

In [10]:
bert_tokenizer_full = BertTokenizer.from_pretrained("bert-base-uncased")

bert_encodings_full = bert_tokenizer_full(
    list(df_train_full["text"]), truncation=True, padding=True, max_length=256
)
bert_labels_full = torch.tensor(list(df_train_full["target"]))

(
    bert_input_ids_train_full,
    bert_input_ids_valid_full,
    bert_token_type_ids_train_full,
    bert_token_type_ids_valid_full,
    bert_attention_mask_train_full,
    bert_attention_mask_valid_full,
    bert_y_train_full,
    bert_y_valid_full,
) = train_test_split(
    bert_encodings_full["input_ids"],
    bert_encodings_full["token_type_ids"],
    bert_encodings_full["attention_mask"],
    bert_labels_full,
    test_size=0.10,
    stratify=bert_labels_full,
    random_state=0,
)

bert_train_encodings_full = {
    "input_ids": torch.tensor(bert_input_ids_train_full),
    "token_type_ids": torch.tensor(bert_token_type_ids_train_full),
    "attention_mask": torch.tensor(bert_attention_mask_train_full),
}

bert_valid_encodings_full = {
    "input_ids": torch.tensor(bert_input_ids_valid_full),
    "token_type_ids": torch.tensor(bert_token_type_ids_valid_full),
    "attention_mask": torch.tensor(bert_attention_mask_valid_full),
}

bert_train_dataset_full = Dataset.from_dict(
    {
        "input_ids": bert_train_encodings_full["input_ids"],
        "attention_mask": bert_train_encodings_full["attention_mask"],
        "labels": bert_y_train_full,
    }
)

bert_valid_dataset_full = Dataset.from_dict(
    {
        "input_ids": bert_valid_encodings_full["input_ids"],
        "attention_mask": bert_valid_encodings_full["attention_mask"],
        "labels": bert_y_valid_full,
    }
)

bert_test_encodings_full = bert_tokenizer_full(
    list(df_test_full["text"]), truncation=True, padding=True, max_length=256
)

bert_test_encodings_full = {
    key: torch.tensor(val) for key, val in bert_test_encodings_full.items()
}

bert_test_dataset_full = Dataset.from_dict(
    {
        "input_ids": bert_test_encodings_full["input_ids"],
        "attention_mask": bert_test_encodings_full["attention_mask"],
    }
)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\cauchepy\Code\.venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cauchepy\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

##### _Save HuggingFace BERT Tokenizer_

In [22]:
bert_tokenizer_full.push_to_hub("yanncauchepin/kaggle_disastertweets_bert_tokenizer")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/yanncauchepin/kaggle_disastertweets_bert_tokenizer/commit/b088a0b7d4a0114a4575c00d7aa16df5f4a1e425', commit_message='Upload tokenizer', commit_description='', oid='b088a0b7d4a0114a4575c00d7aa16df5f4a1e425', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yanncauchepin/kaggle_disastertweets_bert_tokenizer', endpoint='https://huggingface.co', repo_type='model', repo_id='yanncauchepin/kaggle_disastertweets_bert_tokenizer'), pr_revision=None, pr_num=None)

### **BERT Training from scratch**

##### _BERT Transformers + local save_

In [ ]:
bert_model_full = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2
)

training_args = TrainingArguments(
    hub_model_id="yanncauchepin/kaggle_disastertweets_bert_model",
    output_dir="./bert_results",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./bert_logs",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
)

bert_trainer_full = Trainer(
    model=bert_model_full,
    args=training_args,
    train_dataset=bert_train_dataset_full,
    eval_dataset=bert_valid_dataset_full,
)

bert_trainer_full.train()

bert_trainer_full.save_model("bert_model")  # Local

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/25695 [00:00<?, ?it/s]

{'loss': 0.7512, 'grad_norm': 7.5856194496154785, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 0.7962, 'grad_norm': 11.360649108886719, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 0.7241, 'grad_norm': 8.797967910766602, 'learning_rate': 3e-06, 'epoch': 0.02}
{'loss': 0.7007, 'grad_norm': 7.040682792663574, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.02}
{'loss': 0.7179, 'grad_norm': 6.32147741317749, 'learning_rate': 5e-06, 'epoch': 0.03}
{'loss': 0.7423, 'grad_norm': 5.900732040405273, 'learning_rate': 6e-06, 'epoch': 0.04}
{'loss': 0.6424, 'grad_norm': 5.651366233825684, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.04}
{'loss': 0.6589, 'grad_norm': 4.722504615783691, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.05}
{'loss': 0.6378, 'grad_norm': 15.316505432128906, 'learning_rate': 9e-06, 'epoch': 0.05}
{'loss': 0.6683, 'grad_norm': 8.016026496887207, 'learning_rate': 1e-05, 'epoch': 0.06}
{'loss': 0.5659, 'grad_norm': 20.188

  0%|          | 0/191 [00:00<?, ?it/s]

{'eval_loss': 0.4446669816970825, 'eval_runtime': 3.7409, 'eval_samples_per_second': 203.692, 'eval_steps_per_second': 51.057, 'epoch': 1.0}
{'loss': 0.4429, 'grad_norm': 5.398904800415039, 'learning_rate': 4.757888469934511e-05, 'epoch': 1.0}
{'loss': 0.2999, 'grad_norm': 2.588857889175415, 'learning_rate': 4.7559039491962695e-05, 'epoch': 1.01}
{'loss': 0.3847, 'grad_norm': 7.657522678375244, 'learning_rate': 4.753919428458027e-05, 'epoch': 1.02}
{'loss': 0.4099, 'grad_norm': 0.7874215841293335, 'learning_rate': 4.751934907719786e-05, 'epoch': 1.02}
{'loss': 0.7408, 'grad_norm': 0.6209981441497803, 'learning_rate': 4.749950386981544e-05, 'epoch': 1.03}
{'loss': 0.3536, 'grad_norm': 0.8089695572853088, 'learning_rate': 4.747965866243302e-05, 'epoch': 1.03}
{'loss': 0.3451, 'grad_norm': 1.189097285270691, 'learning_rate': 4.745981345505061e-05, 'epoch': 1.04}
{'loss': 0.1993, 'grad_norm': 0.28516486287117004, 'learning_rate': 4.743996824766819e-05, 'epoch': 1.04}
{'loss': 0.5787, 'grad

...

{'eval_loss': 0.8925934433937073, 'eval_runtime': 3.9643, 'eval_samples_per_second': 192.215, 'eval_steps_per_second': 48.18, 'epoch': 13.0}
{'loss': 0.7781, 'grad_norm': 0.2417176067829132, 'learning_rate': 6.796983528477872e-06, 'epoch': 13.0}
{'loss': 0.3167, 'grad_norm': 0.6455737352371216, 'learning_rate': 6.777138321095455e-06, 'epoch': 13.01}
{'loss': 0.3501, 'grad_norm': 9.157003402709961, 'learning_rate': 6.757293113713039e-06, 'epoch': 13.01}
{'loss': 0.3588, 'grad_norm': 0.6108669638633728, 'learning_rate': 6.7374479063306215e-06, 'epoch': 13.02}
{'loss': 0.3322, 'grad_norm': 0.33175191283226013, 'learning_rate': 6.717602698948204e-06, 'epoch': 13.02}
{'loss': 0.1122, 'grad_norm': 0.29274579882621765, 'learning_rate': 6.697757491565787e-06, 'epoch': 13.03}
{'loss': 0.4123, 'grad_norm': 0.38391467928886414, 'learning_rate': 6.67791228418337e-06, 'epoch': 13.04}
{'loss': 0.3997, 'grad_norm': 8.745067596435547, 'learning_rate': 6.658067076800953e-06, 'epoch': 13.04}
{'loss': 0.

  0%|          | 0/191 [00:00<?, ?it/s]

{'eval_loss': 0.8871231079101562, 'eval_runtime': 4.0017, 'eval_samples_per_second': 190.418, 'eval_steps_per_second': 47.73, 'epoch': 14.0}
{'loss': 0.576, 'grad_norm': 8.563016891479492, 'learning_rate': 3.383607858702124e-06, 'epoch': 14.0}
{'loss': 0.3593, 'grad_norm': 9.10401439666748, 'learning_rate': 3.3637626513197067e-06, 'epoch': 14.01}
{'loss': 0.2642, 'grad_norm': 0.2557811737060547, 'learning_rate': 3.3439174439372895e-06, 'epoch': 14.02}
{'loss': 0.2553, 'grad_norm': 0.22549816966056824, 'learning_rate': 3.324072236554872e-06, 'epoch': 14.02}
{'loss': 0.2241, 'grad_norm': 8.994091987609863, 'learning_rate': 3.304227029172455e-06, 'epoch': 14.03}
{'loss': 0.3803, 'grad_norm': 9.014593124389648, 'learning_rate': 3.2843818217900377e-06, 'epoch': 14.03}
{'loss': 0.2489, 'grad_norm': 0.2740814983844757, 'learning_rate': 3.2645366144076205e-06, 'epoch': 14.04}
{'loss': 0.1246, 'grad_norm': 0.24475745856761932, 'learning_rate': 3.2446914070252034e-06, 'epoch': 14.05}
{'loss': 0.

  0%|          | 0/191 [00:00<?, ?it/s]

{'eval_loss': 0.8901631236076355, 'eval_runtime': 3.8752, 'eval_samples_per_second': 196.636, 'eval_steps_per_second': 49.288, 'epoch': 15.0}
{'train_runtime': 3074.0158, 'train_samples_per_second': 33.43, 'train_steps_per_second': 8.359, 'train_loss': 0.4231529526509624, 'epoch': 15.0}


##### _Save HugggingFace trained BERT Transformers_ 

In [ ]:
bert_trainer_full.push_to_hub()

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/yanncauchepin/kaggle_disastertweets_bert_model/commit/76a1c6e651da0749f6b880ac164db12f8a723e56', commit_message='End of training', commit_description='', oid='76a1c6e651da0749f6b880ac164db12f8a723e56', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yanncauchepin/kaggle_disastertweets_bert_model', endpoint='https://huggingface.co', repo_type='model', repo_id='yanncauchepin/kaggle_disastertweets_bert_model'), pr_revision=None, pr_num=None)

### **BERT Loading from pretrained**

In [ ]:
bert_model_full = BertForSequenceClassification.from_pretrained("yanncauchepin/kaggle_disastertweets_bert_model")

training_args = TrainingArguments(
    hub_model_id="yanncauchepin/kaggle_disastertweets_bert_model",
    output_dir="./bert_results",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./bert_logs",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
)

bert_trainer_full = Trainer(
    model=bert_model_full,
    args=training_args,
    train_dataset=bert_train_dataset_full,
    eval_dataset=bert_valid_dataset_full,
)

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

c:\Users\cauchepy\Code\.venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cauchepy\.cache\huggingface\hub\models--yanncauchepin--kaggle_disastertweets_bert_model. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

### **BERT Assessment**

In [14]:
bert_predictions_full = bert_trainer_full.predict(bert_valid_dataset_full)
bert_logits_full = bert_predictions_full.predictions
bert_y_pred_full = np.argmax(bert_logits_full, axis=1)

bert_trainer_full_assessement = evaluate_classifier(
    bert_y_valid_full.numpy(), bert_y_pred_full
)
display(bert_trainer_full_assessement[0])
display(bert_trainer_full_assessement[1])

  0%|          | 0/191 [00:00<?, ?it/s]

,Value
F1 Score,0.844186
Precision,0.845259
Recall,0.845144


,Predicted Negative,Predicted Positive
Actual Negative,389,46
Actual Positive,72,255


### **BERT Test Predictions**

##### _BERT Test Predictions_

In [58]:
bert_test_predictions_full = bert_trainer_full.predict(bert_test_dataset_full)
bert_test_logits_full = bert_test_predictions_full.predictions
bert_test_y_pred_full = np.argmax(bert_test_logits_full, axis=1)

bert_test_submission_full = pd.DataFrame(
    {"id": df_test_full.index, "target": bert_test_y_pred_full.flatten()}
)

  0%|          | 0/816 [00:00<?, ?it/s]

##### _Save HuggingFace BERT Test Predictions_

In [59]:
hf_bert_test_submission_full = Dataset.from_pandas(bert_test_submission_full)
hf_bert_test_submission_full.push_to_hub(
    "yanncauchepin/kaggle_disastertweets_bert_submission_df"
)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/299 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/yanncauchepin/kaggle_disastertweets_bert_submission_df/commit/ba25325ed6dd86d8731491af4fe0bf881995bf99', commit_message='Upload dataset', commit_description='', oid='ba25325ed6dd86d8731491af4fe0bf881995bf99', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/yanncauchepin/kaggle_disastertweets_bert_submission_df', endpoint='https://huggingface.co', repo_type='dataset', repo_id='yanncauchepin/kaggle_disastertweets_bert_submission_df'), pr_revision=None, pr_num=None)